# Diversification and Portfolio Volatility

This notebook demonstrates how adding uncorrelated assets (orthogonal return streams) affects portfolio volatility.

We start with a **1-asset portfolio** and gradually add assets up to **20 assets**, observing how diversification reduces risk.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Function to Generate Orthogonal Assets

We define a function to generate an asset that is **uncorrelated (orthogonal)** to all previously existing assets.

In [2]:
def generate_orthogonal_asset(existing_assets, mean, std_dev, size):
    """
    Generate a new orthogonal asset relative to an existing portfolio.
    """
    new_asset = np.random.normal(loc=mean, scale=std_dev, size=size)
    
    for asset in existing_assets:
        dot_product = np.dot(asset, new_asset) / np.dot(asset, asset)
        new_asset -= dot_product * asset
    
    new_asset = new_asset - np.mean(new_asset) + mean
    new_asset = new_asset * (std_dev / np.std(new_asset))
    return new_asset

## Step 1: Create a Single Asset Portfolio

We begin with one asset that follows a normal distribution.

In [3]:
np.random.seed(42)
mean_daily_return = 0.0001
daily_volatility = 0.004
num_days = 25000

primary_returns = np.random.normal(loc=mean_daily_return, scale=daily_volatility, size=num_days)
primary_cum_returns = (1 + primary_returns).cumprod()

plt.plot(primary_cum_returns, label="1 Asset")
plt.yscale("log")
plt.title("Cumulative Returns - 1 Asset")
plt.xlabel("Days")
plt.ylabel("Wealth (Log Scale)")
plt.legend()
plt.show()

## Step 2: Add a Second Asset

Now, we add a second asset that is uncorrelated with the first.

In [4]:
second_asset = generate_orthogonal_asset([primary_returns], mean_daily_return, daily_volatility, num_days)
weights_2 = [0.5, 0.5]
two_asset_portfolio = weights_2[0] * primary_returns + weights_2[1] * second_asset
two_asset_cum_returns = (1 + two_asset_portfolio).cumprod()

plt.plot(primary_cum_returns, label="1 Asset", linewidth=0.5)
plt.plot(two_asset_cum_returns, label="2 Assets", linewidth=1.5)
plt.yscale("log")
plt.title("Cumulative Returns - 2 Assets")
plt.xlabel("Days")
plt.ylabel("Wealth (Log Scale)")
plt.legend()
plt.show()

## Step 3: Add a Third Asset

Adding a third uncorrelated asset further reduces portfolio volatility.

In [5]:
third_asset = generate_orthogonal_asset([primary_returns, second_asset], mean_daily_return, daily_volatility, num_days)
weights_3 = [1/3, 1/3, 1/3]
three_asset_portfolio = weights_3[0] * primary_returns + weights_3[1] * second_asset + weights_3[2] * third_asset
three_asset_cum_returns = (1 + three_asset_portfolio).cumprod()

plt.plot(primary_cum_returns, label="1 Asset", linewidth=0.5)
plt.plot(two_asset_cum_returns, label="2 Assets", linewidth=0.5)
plt.plot(three_asset_cum_returns, label="3 Assets", linewidth=1.5)
plt.yscale("log")
plt.title("Cumulative Returns - 3 Assets")
plt.xlabel("Days")
plt.ylabel("Wealth (Log Scale)")
plt.legend()
plt.show()

## Step 4: Generalizing to 10, 15, and 20 Assets

Now, we automate the process and observe how volatility changes.

In [6]:
assets = [primary_returns, second_asset, third_asset]
results = {"Number of Assets": [1, 2, 3], "Annual Volatility": []}

for num_assets in [10, 15, 20]:
    while len(assets) < num_assets:
        new_asset = generate_orthogonal_asset(assets, mean_daily_return, daily_volatility, num_days)
        assets.append(new_asset)

    weights = np.ones(len(assets)) / len(assets)
    portfolio_returns = sum(weight * asset for weight, asset in zip(weights, assets))
    results["Number of Assets"].append(num_assets)
    results["Annual Volatility"].append(portfolio_returns.std() * np.sqrt(252))

df = pd.DataFrame(results)
plt.plot(df["Number of Assets"], df["Annual Volatility"], marker='o')
plt.xlabel("Number of Assets")
plt.ylabel("Annualized Volatility")
plt.title("Portfolio Volatility vs Number of Assets")
plt.grid()
plt.show()

df